In [78]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from linearmodels import PanelOLS

df = pd.read_stata('mpd2018.dta')
df['year'] = df['year'].astype(int)
df = df.loc[df['year']>1950]

table = pd.pivot_table(df,index=['countrycode','year'],values=['cgdppc','rgdpnapc'])

for i in df['countrycode'].unique():
    table.loc[i,'g'] = np.log(table.loc[i,'rgdpnapc']).diff(5).rolling(5).mean().values
    

table.reset_index(inplace=True)
table = table[table['year']%5==0].dropna(subset=['g'])
data = table.set_index(['countrycode','year']).dropna()
    
data.head(10)

cgdppc  rgdpnapc         g
countrycode year                            
AFG         1960  2744.0    2744.0  0.058920
            1965  2670.0    2670.0  0.004578
            1970  2632.0    2632.0 -0.015458
            1975  2225.0    2225.0 -0.176975
            1980  2108.0    2108.0 -0.021972
            1985  2700.0    2700.0  0.199447
            1990  1992.0    1992.0 -0.136888
            1995  1101.0    1101.0 -0.642467
            2000   735.0     735.0 -0.348013
            2005  1187.0    1187.0  0.173445

In [79]:
import statsmodels.formula.api as smf

res = smf.ols('g ~ np.log(cgdppc)',data=data).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      g   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     60.30
Date:                Mon, 20 Jan 2020   Prob (F-statistic):           1.32e-14
Time:                        21:52:21   Log-Likelihood:                 702.83
No. Observations:                1893   AIC:                            -1402.
Df Residuals:                    1891   BIC:                            -1391.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.1248      0.029     -4.358      0.000      -0.181      -0.069
np.log(cgdppc)     0.0257      0.003      7.766      0.000       0.019       0.032
==============================================================================
Omnibus:                      551.479   Durbin-Watson:                   1.192
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6248.091
Skew:                          -1.034   Prob(JB):                         0.00
Kurtosis:                      11.657   Cond. No.                         65.6
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [80]:
mod = PanelOLS.from_formula('g ~ 1 + np.log(cgdppc) + EntityEffects',data=data).fit()
mod.summary

Dep. Variable:,g,R-squared:,0.0277
Estimator:,PanelOLS,R-squared (Between):,-0.3190
No. Observations:,1893,R-squared (Within):,0.0277
Date:,"Mon, Jan 20 2020",R-squared (Overall):,-0.0063
Time:,21:52:31,Log-likelihood,858.61
Cov. Estimator:,Unadjusted,,
,,F-statistic:,49.127
Entities:,169,P-value,0.0000
Avg Obs:,11.201,Distribution:,"F(1,1723)"
Min Obs:,3.0000,,
Max Obs:,12.000,F-statistic (robust):,49.127
